# Get Bibliographic Information
This notebooks finds the bibliographic information (Author, Co-Authors, Title, DOI, Date, Publisher Name, Print ISBN, Online ISBN and Abstract) from any paper from the given CIAA-Conference (URL).
###### written by Nils Dyck, 30.09.2022

In [32]:
# Downloads the html-Code of the given URL and cooks the soup

import urllib.request, urllib.error, urllib.parse, re

from bs4 import BeautifulSoup

url = input('Enter URL to download: ')
#url = 'https://link.springer.com/book/10.1007/978-3-031-07469-1'

response = urllib.request.urlopen(url)
webContent = response.read().decode('UTF-8')

soup = BeautifulSoup(webContent, 'html.parser')

In [33]:
import re #for using regular expressions to find paper titles
titles = soup.find_all(href = re.compile('/chapter/')) #collects titles

In [34]:
#create urls for paper-sites
urls = []
for j in range(1,len(titles)+1):
    urls.append(url+'_'+str(j))
for k in range(len(urls)):
    urls[k] = urls[k].replace('book', 'chapter')
#urls

paper_sites = []
for l in range(len(urls)):
    response = urllib.request.urlopen(urls[l])
    htmlCode = response.read().decode('UTF-8')
    paper_sites.append(BeautifulSoup(htmlCode, 'html.parser'))


In [35]:
#creating csv-file

#Only use the two following lines if you want to overwrite the exisiting data
#with open('collaboration.csv', 'w') as f:
#    f.write('Author;Co-Authors;Title;DOI;Date;Publisher Name;Print ISBN;Online ISBN; Abstract\n')

#finding bibliographic information for every paper from the given conference
for site in paper_sites:
    #finding authors and saving them in a list called 'author_list'
    authors = site.find_all(href = re.compile('#auth.*'))
    author_list = []
    for author in authors:
        for string in author.stripped_strings:
            #print(repr(string))
            author_list.append(string)
    #author_list
    co_authors = []
    for i in range(len(author_list)):
        co_authors.append([])
        for j in range(len(author_list)):
            if i != j:
                co_authors[i].append(author_list[j])
    #print(co_authors)
    co_author_string = []
    for k in range(len(co_authors)):
        co_author_string.append('')
        for l in range(len(co_authors[k])):
            co_author_string[k]+= (co_authors[k][l]+', ')
        co_author_string[k] = co_author_string[k][:-2]



    #finding the abstract and saving it in a string calle 'abstract_string'
    abstract = site.find(class_ = 'c-article-section__content')
    abstract_string = abstract.find('p')
    #Cleaning up the abstract a little bit
    abstract_string = str(abstract_string).replace(";",".") #for the csv-file there must not be any semicola in the abstracts
    abstract_string = re.sub("<.*?>","",abstract_string) #no html-tag should survive this action.
    abstract_string = abstract_string.replace("\n","")
    #abstract_string

    #finding other bibliographic information
    bi = site.find_all(class_ = 'c-bibliographic-information__value')
    
    #finding venue
    venue = site.find(class_ = 'c-chapter-book-details right-arrow')
    venue = repr(venue.stripped_strings)
    #venue

    #finding title
    title = site.title
    title = str(title)
    title = title.replace('\xa0', ' ')
    title = re.sub("<.*?>", "", title) #no html-tag should survive this action.
    title = title.replace(' | SpringerLink', '')
    #title

    #writing the output-file
    with open('collaboration.csv', 'a') as f:
        for i in range(len(author_list)):
            f.write(author_list[i]+';')
            f.write(str(co_author_string[i])+';')#clean form
            #f.write(str(co_authors[i])+';')#list form
            f.write(title+';')
            f.write(bi[0].string+'; '+bi[1].string+'; '+bi[2].string+'; '+bi[3].string+'; '+bi[4].string+';')
            f.write(abstract_string)
            f.write('\n')

IndexError: list index out of range